
poetry run python -m uvicorn tplexity.retriever.app:app --host 0.0.0.0 --port 8010 --reload

In [1]:
import logging
import sys
import time
from pathlib import Path
from typing import Any

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")

print(f"📁 Корневая директория проекта: {project_root}")

📁 Корневая директория проекта: /srv/nlp1/eval_dir/T-bank_NLP_


In [6]:
from tplexity.retriever.sparse_embedding import lemmatize_text
print(lemmatize_text("Сегодня произошла важная новость, т банк опубликовал квартальный отчет"))


ImportError: cannot import name 'lemmatize_text' from 'tplexity.retriever.sparse_embedding' (/srv/nlp1/eval_dir/T-bank_NLP_/src/tplexity/retriever/sparse_embedding.py)

In [3]:
# Инициализация сервиса
import httpx

# Проверка доступности Retriever API перед инициализацией
retriever_url = "http://localhost:8010"
try:
    with httpx.Client(timeout=5.0) as client:
        response = client.get(f"{retriever_url}/health")
        if response.status_code == 200:
            print(f"✅ Retriever API доступен: {retriever_url}")
        else:
            print(f"⚠️ Retriever API вернул статус {response.status_code}")
except Exception as e:
    print(f"❌ ОШИБКА: Retriever API недоступен по адресу {retriever_url}")
    print(f"   Ошибка: {e}")
    print("\n   Пожалуйста, запустите Retriever сервис:")
    print("   python -m uvicorn tplexity.retriever.app:app --host 0.0.0.0 --port 8010 --reload")
    raise

# Инициализация сервиса
service = GenerationService()

print("✅ GenerationService инициализирован")
print(f"Retriever API URL: {service.retriever_client.base_url}")

2025-11-16 22:07:30,523 - httpx - INFO - HTTP Request: GET http://localhost:8010/health "HTTP/1.1 200 OK"
2025-11-16 22:07:30,524 - tplexity.generation.generation_service - INFO - 🔄 [generation_service] Инициализация сервиса генерации
2025-11-16 22:07:30,525 - tplexity.generation.generation_service - INFO - 🔄 [retriever_client] Инициализирован клиент для http://localhost:8010
2025-11-16 22:07:30,525 - tplexity.llm_client.client - INFO - 🔄 [llm_client] Инициализация LLM клиента: model=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ, base_url=http://195.209.210.28:8000/v1
2025-11-16 22:07:30,532 - tplexity.llm_client.client - INFO - ✅ [llm_client] LLM клиент инициализирован
2025-11-16 22:07:30,532 - tplexity.generation.generation_service - INFO - ✅ [generation_service] Сервис генерации инициализирован: provider=qwen


✅ Retriever API доступен: http://localhost:8010
✅ GenerationService инициализирован
Retriever API URL: http://localhost:8010


In [4]:
# Тестовые запросы
TEST_QUERIES = [
    "Что такое банковский вклад?",
    "Как открыть счет в банке?",
    "В чем разница между дебетовой и кредитной картой?",
    "Какие условия для получения ипотечного кредита?",
]

# Параметры для тестирования
TEST_PARAMS = {
    "top_k": 5,
    "use_rerank": False,
    "temperature": 0.7,
    "max_tokens": 1000,
}

print("Тестовые запросы:")
for i, query in enumerate(TEST_QUERIES, 1):
    print(f"{i}. {query}")
print(f"\nПараметры генерации: {TEST_PARAMS}")

Тестовые запросы:
1. Что такое банковский вклад?
2. Как открыть счет в банке?
3. В чем разница между дебетовой и кредитной картой?
4. Какие условия для получения ипотечного кредита?

Параметры генерации: {'top_k': 5, 'use_rerank': False, 'temperature': 0.7, 'max_tokens': 1000}


In [5]:
async def test_generation(
    service: GenerationService, query: str, llm_provider: str | None = None, **kwargs
) -> dict[str, Any]:
    """
    Тестирует генерацию ответа через GenerationService

    Args:
        service: Экземпляр GenerationService
        query: Запрос пользователя
        llm_provider: Провайдер LLM (если None, используется из config)
        **kwargs: Дополнительные параметры (top_k, use_rerank, temperature, max_tokens)

    Returns:
        Словарь с результатами тестирования
    """
    result = {
        "query": query,
        "llm_provider": llm_provider,
        "status": "unknown",
        "answer": None,
        "doc_ids": [],
        "metadatas": [],
        "time": 0,
        "error": None,
    }

    start_time = time.time()

    try:
        print(f"\n{'='*60}")
        print(f"🔍 Запрос: {query}")
        if llm_provider:
            print(f"🤖 Провайдер LLM: {llm_provider}")
        print(f"⚙️  Параметры: {kwargs}")

        answer, doc_ids, metadatas = await service.generate(query=query, llm_provider=llm_provider, **kwargs)

        elapsed_time = time.time() - start_time
        result["status"] = "success"
        result["answer"] = answer
        result["doc_ids"] = doc_ids
        result["metadatas"] = metadatas
        result["time"] = elapsed_time

        print(f"✅ Успешно за {elapsed_time:.2f}с")
        print(f"📄 Найдено источников: {len(doc_ids)}")
        print(f"💬 Длина ответа: {len(answer)} символов")
        print(f"\n📝 Ответ:\n{answer[:300]}...")
        if doc_ids:
            print("\n📚 Источники:")
            for i, (doc_id, metadata) in enumerate(zip(doc_ids, metadatas, strict=False), 1):
                print(f"  {i}. Doc ID: {doc_id}")
                if metadata:
                    print(f"     Metadata: {metadata}")

    except ValueError as e:
        elapsed_time = time.time() - start_time
        result["status"] = "validation_error"
        result["error"] = str(e)
        result["time"] = elapsed_time
        print(f"❌ Ошибка валидации: {e}")

    except Exception as e:
        elapsed_time = time.time() - start_time
        result["status"] = "error"
        result["error"] = str(e)
        result["time"] = elapsed_time

        # Детальная информация об ошибке
        error_msg = str(e)
        print(f"❌ Ошибка: {error_msg}")

        # Если это ошибка Retriever API, даем рекомендации
        if "Retriever API" in error_msg or "500" in error_msg:
            print("\n💡 Рекомендации:")
            print("   1. Проверьте логи Retriever API сервиса")
            print("   2. Убедитесь, что Retriever API запущен: curl http://localhost:8010/health")
            print("   3. Проверьте настройки подключения к Qdrant в retriever/.env")
            print("   4. Убедитесь, что модели (embeddings, reranker) загружены корректно")
            print("   5. Проверьте, что в базе есть документы для поиска")

    return result

In [6]:
# Тест 1: Базовая генерация с провайдером по умолчанию
result1 = await test_generation(service=service, query=TEST_QUERIES[0], **TEST_PARAMS)

2025-11-16 22:07:38,218 - tplexity.generation.generation_service - INFO - 🔄 [generation_service] Запрос без указания llm_provider, используется провайдер по умолчанию: qwen
2025-11-16 22:07:38,219 - tplexity.generation.generation_service - INFO - 🔄 [generation_service] Начало генерации для запроса: Что такое банковский вклад?...



🔍 Запрос: Что такое банковский вклад?
⚙️  Параметры: {'top_k': 5, 'use_rerank': False, 'temperature': 0.7, 'max_tokens': 1000}


2025-11-16 22:08:01,110 - httpx - INFO - HTTP Request: POST http://localhost:8010/retriever/search "HTTP/1.1 200 OK"
2025-11-16 22:08:01,134 - tplexity.generation.generation_service - INFO - ✅ [generation_service] Найдено 5 релевантных документов
2025-11-16 22:08:01,137 - tplexity.generation.generation_service - INFO - ✅ [generation_service] Использование провайдера по умолчанию: qwen (модель: QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ, base_url: http://195.209.210.28:8000/v1)
2025-11-16 22:08:01,138 - tplexity.generation.generation_service - INFO - 🔄 [generation_service] Генерация ответа через LLM провайдер=qwen, модель=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ
2025-11-16 22:08:01,139 - tplexity.llm_client.client - INFO - 🔄 [llm_client] Отправка запроса к LLM: model=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ, base_url=http://195.209.210.28:8000/v1
2025-11-16 22:08:10,874 - httpx - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 22:08:10,90

✅ Успешно за 32.69с
📄 Найдено источников: 5
💬 Длина ответа: 3103 символов

📝 Ответ:
**Подробный ответ**

Банковский вклад — это форма хранения денежных средств в банке, при которой клиент передает банку определённую сумму денег на определённый срок с целью получения дохода в виде процентов. Вклады являются одним из наиболее распространённых и безопасных инструментов для сохранения ...

📚 Источники:
  1. Doc ID: 6f4fadb3-df2d-4d34-bd88-f4bea9b1d8df
     Metadata: {'doc_id': '1498653424_23333', 'message_id': 23333, 'channel_id': 1498653424, 'date': '2025-11-07T08:30:01+00:00', 'link': 'https://t.me/selfinvestor/23333'}
  2. Doc ID: 550c37ee-9c2f-41e4-8775-afa22c660644
     Metadata: {'doc_id': '1498653424_23301', 'message_id': 23301, 'channel_id': 1498653424, 'date': '2025-11-05T19:09:03+00:00', 'link': 'https://t.me/selfinvestor/23301'}
  3. Doc ID: 598be75f-02a7-484f-a4b4-137b8468dabd
     Metadata: {'doc_id': '1680634724_3070', 'message_id': 3070, 'channel_id': 1680634724, 'date': '20

In [7]:
print(result1["answer"])

**Подробный ответ**

Банковский вклад — это форма хранения денежных средств в банке, при которой клиент передает банку определённую сумму денег на определённый срок с целью получения дохода в виде процентов. Вклады являются одним из наиболее распространённых и безопасных инструментов для сохранения и прироста капитала. Вклады могут быть как физическим, так и юридическим лицам, и они подразделяются на различные типы в зависимости от условий, срока, ставки и других параметров.

В контексте предоставленных документов, банк «Дом.РФ» запустил новый вклад под названием «Могучий» со ставкой 35% годовых, который доступен с 7 ноября 2025 года и действует до 17 ноября 2025 года. Этот вклад можно открыть на сумму от 10 тыс. до 100 тыс. рублей на срок от 31 дня до 17 ноября 2025 года включительно [1]. Это предложение доступно только для первого вклада, открытого через платформу «Финуслуги», что указывает на то, что такой высокий процент может быть ограниченным и временным.

Среди других банков, ср

In [ ]:
# Тест 2: Генерация с указанным провайдером
PROVIDERS = ["qwen", "deepseek"]  # Можно добавить другие провайдеры

for provider in PROVIDERS:
    try:
        result = await test_generation(service=service, query=TEST_QUERIES[0], llm_provider=provider, **TEST_PARAMS)
        print()
    except Exception as e:
        print(f"❌ Ошибка при тестировании провайдера {provider}: {e}\n")

In [ ]:
# Тест 3: Генерация с разными параметрами
test_cases = [
    {"top_k": 3, "use_rerank": True, "temperature": 0.5, "max_tokens": 500},
    {"top_k": 5, "use_rerank": False, "temperature": 0.7, "max_tokens": 1000},
    {"top_k": 10, "use_rerank": True, "temperature": 0.9, "max_tokens": 2000},
]

for i, params in enumerate(test_cases, 1):
    print(f"\n{'='*60}")
    print(f"Тест {i}: Параметры {params}")
    result = await test_generation(service=service, query=TEST_QUERIES[1], **params)
    print()

In [ ]:
# Тест 4: Обработка пустого запроса
print("Тест: Обработка пустого запроса")
try:
    result = await test_generation(service=service, query="", **TEST_PARAMS)
except ValueError as e:
    print(f"✅ Корректно обработана ошибка: {e}")
except Exception as e:
    print(f"❌ Неожиданная ошибка: {e}")

print("\n" + "=" * 60)
print("Тест: Обработка запроса только из пробелов")
try:
    result = await test_generation(service=service, query="   ", **TEST_PARAMS)
except ValueError as e:
    print(f"✅ Корректно обработана ошибка: {e}")
except Exception as e:
    print(f"❌ Неожиданная ошибка: {e}")

In [ ]:
# Тест 5: Генерация для всех тестовых запросов
all_results = []

for query in TEST_QUERIES:
    result = await test_generation(service=service, query=query, **TEST_PARAMS)
    all_results.append(result)
    print()

print("=" * 60)
print("📊 Сводка результатов:")
print("=" * 60)
for i, result in enumerate(all_results, 1):
    status_emoji = "✅" if result["status"] == "success" else "❌"
    print(f"{i}. {status_emoji} {result['query'][:50]}...")
    print(f"   Статус: {result['status']}")
    if result["status"] == "success":
        print(f"   Время: {result['time']:.2f}с")
        print(f"   Источников: {len(result['doc_ids'])}")
        print(f"   Длина ответа: {len(result['answer'])} символов")
    else:
        print(f"   Ошибка: {result['error']}")
    print()

In [ ]:
# Тест 6: Проверка формирования промпта
print("Тест: Проверка формирования промпта")
print("=" * 60)

# Создаем тестовые документы
test_documents = [
    ("doc1", 0.95, "Банковский вклад - это денежные средства, размещенные в банке.", {"channel": "test"}),
    ("doc2", 0.85, "Вклады бывают срочные и до востребования.", {"channel": "test"}),
]

test_query = "Что такое банковский вклад?"
prompt = service._build_user_prompt(test_query, test_documents)

print("Сформированный промпт:")
print("-" * 60)
print(prompt)
print("-" * 60)

In [ ]:
print("✅ Тестирование завершено!")

In [ ]:
import httpx

# Диагностика: Проверка состояния Retriever API
print("=" * 60)
print("🔍 Диагностика Retriever API")
print("=" * 60)

retriever_url = "http://localhost:8010"

# Проверка health endpoint
try:
    with httpx.Client(timeout=5.0) as client:
        health_response = client.get(f"{retriever_url}/health")
        print(f"✅ Health check: {health_response.status_code}")
        print(f"   Ответ: {health_response.json()}")
except Exception as e:
    print(f"❌ Health check не прошел: {e}")

# Проверка root endpoint
try:
    with httpx.Client(timeout=5.0) as client:
        root_response = client.get(f"{retriever_url}/")
        print(f"✅ Root endpoint: {root_response.status_code}")
        if root_response.status_code == 200:
            info = root_response.json()
            print(f"   Сервис: {info.get('service')}")
            print(f"   Версия: {info.get('version')}")
except Exception as e:
    print(f"❌ Root endpoint не доступен: {e}")

# Попытка выполнить простой поиск для диагностики
print("\n" + "=" * 60)
print("🔍 Тестовый поиск для диагностики")
print("=" * 60)

try:
    with httpx.Client(timeout=10.0) as client:
        search_payload = {"query": "тест", "top_k": 1, "top_n": 1, "use_rerank": False}
        search_response = client.post(f"{retriever_url}/retriever/search", json=search_payload, timeout=10.0)
        print(f"Статус: {search_response.status_code}")
        if search_response.status_code == 200:
            data = search_response.json()
            print(f"✅ Поиск успешен, найдено: {data.get('total', 0)} результатов")
        else:
            print("❌ Поиск не удался")
            try:
                error_data = search_response.json()
                print(f"   Детали ошибки: {error_data.get('detail', 'Нет деталей')}")
            except Exception:
                print(f"   Текст ответа: {search_response.text[:500]}")
except Exception as e:
    print(f"❌ Ошибка при тестовом поиске: {e}")
    import traceback

    print(f"   Traceback: {traceback.format_exc()}")